In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test, save, load, get_loss_history
from Models.basic_models import average_model, two_layer_basic, post_process

set_figsize(20, 15)

Training directory found, 36 series
Validation directory found, 6 series
Testing directory found, 10 series


In [3]:
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# "Pixl", "Slice", or "None"
preproc_type = "pixel"

validation = TAVR_3_Frame("__valid", preproc=preproc_type, preload=True)
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc=preproc_type, preload=True)
train_loader = tavr_dataloader(training,batch_size=6, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_layer_basic()
post_proc = post_process(kind=preproc_type).to(device=device)
loss_fn = batch_l2_loss()

# CHANGE TO NAME OF JUPYTER NOTEBOOK
model_name = "Model 5 (Basic+PixelNorm) R1.1"

using device: cuda


In [4]:
learning_rate = 3e-3
momentum = 0.90
reg = 1e-7

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=reg, nesterov=True)

In [5]:
LOAD = True
iteration_num = 0

if LOAD:
    load(model_name, iteration_num, model, optimizer)
    loss_history = get_loss_history(model_name)
    model.to(device=device)
else:
    loss_history = None

model loaded from model_checkpoints/Model 5 (Basic+PixelNorm) R1.1/Model 5 (Basic+PixelNorm) R1.1-0


In [15]:
optimizer.cuda()


AttributeError: 'SGD' object has no attribute 'cuda'

In [17]:
for state in optimizer.state.values():
    for k, v in state.items():
        state[k] = v.cuda()

In [ ]:
# If multiple GPU
model = nn.DataParallel(model)

In [18]:
train(model, post_proc, optimizer, train_loader, val_loader, loss_fn, device, 
         model_name, loss_history,
          epochs=1, print_every=30, print_level=4, lr_decay=1)


Iteration 0, loss = 2.6620, corrected loss = 1238.6766
Validation loss 907.2852 over 81 frames
model saved to model_checkpoints/Model 5 (Basic+PixelNorm) R1.1/Model 5 (Basic+PixelNorm) R1.1-0
conv_a1.weight,   	norm: 4.2612e+00, 	update norm: 2.9970e-02 	Update/norm: 7.0333e-03
conv_a1.bias,   	norm: 2.4315e-01, 	update norm: 5.4741e-03 	Update/norm: 2.2514e-02
conv_b1.weight,   	norm: 3.8789e+00, 	update norm: 5.2326e-02 	Update/norm: 1.3490e-02
conv_b1.bias,   	norm: 3.2159e-01, 	update norm: 7.9045e-03 	Update/norm: 2.4579e-02
final.weight,   	norm: 2.0066e+00, 	update norm: 3.7283e-02 	Update/norm: 1.8580e-02
final.bias,   	norm: 2.1738e-01, 	update norm: 7.8349e-03 	Update/norm: 3.6042e-02

.

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, post_proc, val_loader, loss_fn, device)
test(ave_model, post_proc, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [val_seq[0][0]]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [ave_frame[0][0],  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)